## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_clean = application_df.copy()
application_clean.drop(columns=["EIN","NAME","SPECIAL_CONSIDERATIONS"], axis=1, inplace=True)
application_clean.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,142590,1


In [3]:
# Determine the number of unique values in each column.
application_clean.nunique()

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
USE_CASE               5
ORGANIZATION           4
STATUS                 2
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [4]:
amt = application_clean["ASK_AMT"]

In [5]:
amt.mean()

2769198.6820607013

In [6]:
amt.median()

5000.0

In [7]:
amt_values = amt.value_counts()
amt.sort_values()

0              5000
20358          5000
20356          5000
20355          5000
20354          5000
            ...    
31337    3391919220
24795    4653011914
33678    5591584994
34222    8556638692
33175    8597806340
Name: ASK_AMT, Length: 34299, dtype: int64

In [8]:
bins = [0, 5001, 49999, 99999, 249999, 999999, 90000000000]
labels = [" 5K or less ", " 5K-50K ", " 50K-100k", " 100k-250k ", " 250k-1M ", "Over 1M" ]

In [9]:
application_clean["ASK_BINS"] = pd.cut(amt, bins, labels=labels)




In [10]:
application_clean.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL,ASK_BINS
0,T10,Independent,C1000,ProductDev,Association,1,0,5000,1,5K or less
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,108590,1,100k-250k
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,5000,0,5K or less
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,6692,1,5K-50K
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,142590,1,100k-250k


In [11]:
application_clean["ASK_BINS"].value_counts()

 5K or less     25399
 5K-50K          2946
Over 1M          1577
 250k-1M         1513
 100k-250k       1443
 50K-100k        1421
Name: ASK_BINS, dtype: int64

In [12]:
application_clean.drop(columns="ASK_AMT", axis=1, inplace=True)

In [13]:
# Create 3 Income Buckets
income_values = application_clean["INCOME_AMT"].value_counts()

In [14]:
income_dict = income_values.to_dict()
income_dict

{'0': 24388,
 '25000-99999': 3747,
 '100000-499999': 3374,
 '1M-5M': 955,
 '1-9999': 728,
 '10000-24999': 543,
 '10M-50M': 240,
 '5M-10M': 185,
 '50M+': 139}

In [15]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
high_income_replace = []
mid_income_replace = []

for key,value in income_dict.items():
  if value < 1000:
    high_income_replace.append(key)
  elif value > 1000 and value < 4000:
    mid_income_replace.append(key)

# Replace in dataframe
for app in high_income_replace:
    application_clean['INCOME_AMT'] = application_clean['INCOME_AMT'].replace(app,"1M+")

for app in mid_income_replace:
    application_clean['INCOME_AMT'] = application_clean['INCOME_AMT'].replace(app,"25000-499999")


application_clean['INCOME_AMT'].value_counts()



0               24388
25000-499999     7121
1M+              2790
Name: INCOME_AMT, dtype: int64

In [16]:
# Look at APPLICATION_TYPE value counts for binning
application_values = application_clean["APPLICATION_TYPE"].value_counts()
application_values

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [17]:
#Compile values into dictionary
application_dict = application_values.to_dict()

In [18]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = []

for key,value in application_dict.items():
  if value < 156:
    application_types_to_replace.append(key)

# Replace in dataframe
for app in application_types_to_replace:
    application_clean['APPLICATION_TYPE'] = application_clean['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_clean['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

In [19]:
# Look at CLASSIFICATION value counts for binning
class_values = application_clean["CLASSIFICATION"].value_counts()
class_values

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [20]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_values_limited = application_clean["CLASSIFICATION"].value_counts()[application_clean["CLASSIFICATION"].value_counts() >1]
class_values_limited

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [21]:
# Convert value counts to dict
class_dict = class_values.to_dict()

In [22]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = []

for key,value in class_dict.items():
  if value < 104:
    classifications_to_replace.append(key)


# Replace in dataframe
for cls in classifications_to_replace:
    application_clean['CLASSIFICATION'] = application_clean['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_clean['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [23]:
# application_clean.drop(columns=["ASK_BINS"], axis=1, inplace=True)

In [24]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummies = pd.get_dummies(application_clean, columns=["ORGANIZATION", "CLASSIFICATION", "APPLICATION_TYPE",\
                                                                 "USE_CASE", "ASK_BINS", "INCOME_AMT","AFFILIATION", "STATUS"])

application_dummies.head()

,IS_SUCCESSFUL,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C1270,CLASSIFICATION_C1700,CLASSIFICATION_C2000,...,INCOME_AMT_1M+,INCOME_AMT_25000-499999,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,STATUS_0,STATUS_1
0,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
1,1,0,1,0,0,0,0,0,0,1,...,1,0,0,0,1,0,0,0,0,1
2,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
3,1,0,0,0,1,0,0,0,0,1,...,1,0,1,0,0,0,0,0,0,1
4,1,0,0,0,1,1,0,0,0,0,...,0,1,0,0,1,0,0,0,0,1


In [25]:
# Split our preprocessed data into our features and target arrays
y = application_dummies["IS_SUCCESSFUL"].values.reshape(-1, 1)
X = application_dummies.copy()
X.drop(columns="IS_SUCCESSFUL", axis=1, inplace=True)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [26]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [27]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#

# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 30
hidden_nodes_layer4 = 100
# hidden_nodes_layer4 = 100
# hidden_nodes_layer4 = 100
# hidden_nodes_layer4 = 100

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# # Third hidden layer
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# # # # Fourth hidden layer
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# # Fourth hidden layer
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# # # Fourth hidden layer
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# # # Fourth hidden layer
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# # # Fourth hidden layer
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3920      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 6,381
Trainable params: 6,381
Non-trainable params: 0
_________________________________________________________________


In [28]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [29]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=200)


Epoch 1/200
804/804 [==============================] - 7s 6ms/step - loss: 0.5665 - accuracy: 0.7211
Epoch 2/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5528 - accuracy: 0.7278
Epoch 3/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5492 - accuracy: 0.7302
Epoch 4/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5469 - accuracy: 0.7311
Epoch 5/200
804/804 [==============================] - 4s 5ms/step - loss: 0.5461 - accuracy: 0.7328
Epoch 6/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5448 - accuracy: 0.7336
Epoch 7/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5440 - accuracy: 0.7337
Epoch 8/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5430 - accuracy: 0.7337
Epoch 9/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5423 - accuracy: 0.7341
Epoch 10/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5417 - accura

In [30]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5704 - accuracy: 0.7283 - 512ms/epoch - 2ms/step
Loss: 0.570357084274292, Accuracy: 0.7282798886299133


In [31]:
# Export our model to HDF5
nn.save(AlphabetSoupCharity_Optimization.h5)



NameError: ignored